In [1]:
!pip install elements
!pip install ninjax
!pip install crafter
!pip install jax
!pip install ale-py

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 201.0 MB/s eta 0:00:00
  Created wheel for elements: filename=elements-3.18.2-py3-none-any.whl size=34831 sha256=c8997f382207b73e1d98d86bbc3d0728c0d6c1a36edb7ee358cd9946ffea84ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-v7mm_gbr/wheels/2d/4d/1c/6fcaebb015630e5f18347bca1e68a977d1661f0c834bb69f11
Successfully built elements

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 323.6 MB/s eta 0:0

In [2]:
!pip install -r ../dreamerv3/requirements.txt -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 635.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 70.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 462.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 30.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 9.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 MB 9.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━

In [3]:
!pip install jax -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 311.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 MB 457.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os
import json
import tqdm
import torch
import pickle
import numpy as np
import pandas as pd
import os
import ninjax as nj
from pathlib import Path
from typing import Tuple, List, Dict, Any
from torch.utils.data import DataLoader
import elements
import jax.numpy as jnp
import ruamel.yaml as yaml

In [2]:
root = Path(os.getcwd()).parent
os.sys.path.append(str(root))
os.sys.path.append(str(root / "dreamerv3"))

In [3]:
# import locals
from models.agent_probe import Agent, sg
from models.rssm_probe import Decoder
from dreamerv3.agent import Agent as AgentOri
from embodied.envs import crafter
from embodied.jax import transform

# Functions

In [4]:
def load_config(argv=None):
    # adaptation from dreamerv3/main.py
    configs = elements.Path('./configs.yaml').read()
    configs = yaml.YAML(typ='safe').load(configs)
    parsed, other = elements.Flags(configs=['defaults']).parse_known(argv)
    config = elements.Config(configs['defaults'])
    for name in parsed.configs:
        config = config.update(configs[name])
    # config = elements.Flags(config).parse(other)
    config = config.update(logdir=(
        config.logdir.format(timestamp=elements.timestamp())))

    if 'JOB_COMPLETION_INDEX' in os.environ:
        config = config.update(replica=int(os.environ['JOB_COMPLETION_INDEX']))
    print('Replica:', config.replica, '/', config.replicas)

    logdir = elements.Path(config.logdir)
    print('Logdir:', logdir)
    print('Run script:', config.script)

    return config

In [5]:
def load_dataset(path='./dataset/crafter', pickle=True) -> Tuple[List[Dict[str, np.ndarray]], List[Dict[str, np.ndarray]]]:
    img_pos = []
    img_epi = []

    stats_files = os.listdir(path)
    path = Path(path)
    print("stats files num: ", len(stats_files))
    # if pickle is True:
    #     with open(path/'pickles/img_pos.pkl', 'rb') as handle:
    #         img_pos = pickle.load(handle)
    #     with open(path/'pickles/img_epi.pkl', 'rb') as handle:
    #         img_epi = pickle.load(handle)
    # else:
    for file in tqdm.tqdm(stats_files):
        if "stats" in file:
            with open(path / file, 'r') as f:
                data = [json.loads(line) for line in f]
    
            for d in data:
                img_pos.append({"image": np.array(d['image']), 'pos': np.array(d['pos'])})
                img_epi.append({"image": np.array(d['image']), 'episode': np.array(d['episode'])})
    print("total length: ", len(img_pos))
    return img_pos, img_epi

In [6]:
def load_agent(agent, path_to_pkl="./models/agent.pkl"):
    path_to_pkl = Path(path_to_pkl)
    with open(path_to_pkl, 'rb') as f:
        if "agent" in str(path_to_pkl):
            weight = pickle.load(f)
        else:
            weight = pickle.load(f)['agent']
    print("Agent loaded from", path_to_pkl)
    print("Agent type:", type(weight))
    print(type(agent))
    agent.load(weight)
    print(type(agent))
    return agent

In [7]:
import inspect
def overwrite_func(agent):
    # overwrite call function in decoder
    new_dec_call = lambda carry, feat, reset, training, single=False: Decoder.__call__(agent.model.dec, carry, feat, reset, training, single=False)
    # print("new_dec_call: ")
    # lines = inspect.getsource(Decoder.__call__)
    agent.model.dec.__call__ = new_dec_call

    # overwrite policy function in agent
    new_agent_policy = lambda carry, obs, mode='train': Agent.policy(agent.model, carry, obs, mode='train')
    agent.model.policy = new_agent_policy

    return agent

In [8]:
def reconstruct_obs_dummy(img, batch=16):
    obs = dict(
        image=img,
        reward=np.stack([np.float32(0) for _ in range(batch)]),
        is_first=np.stack([False]*batch),
        is_last=np.stack([False]*batch),
        is_terminal=np.stack([False]*batch),
    )
    return obs

In [9]:
import jax
def collect_activations(carry, obs, agent: Agent):
    # what is carry?: weight?
    # prevact = previous action
    (enc_carry, dyn_carry, dec_carry, prevact) = carry
    kw = dict(training=False, single=True)
    reset = obs['is_first']

    # editted
    activations = {}

    # what is tokens?
    enc_carry, enc_entry, tokens = agent.enc(enc_carry, obs, reset, **kw)

    # editted
    activations['encoder'] = tokens

    dyn_carry, dyn_entry, feat = agent.dyn.observe(
        dyn_carry, tokens, prevact, reset, **kw)

    # editted
    activations['dynamic'] = feat

    dec_entry = {}
    # if dec_carry:
    dec_carry, dec_entry, recons = agent.dec(dec_carry, feat, reset, **kw)

    # editted
    activations['decoder'] = recons['x']

    # actor: linear transformation of RSSM feature into action space distribution
    sample = lambda xs: jax.tree.map(lambda x: x.sample(nj.seed()), xs)
    policy = agent.pol(agent.feat2tensor(feat), bdims=1)
    act = sample(policy)

    # editted
    activations['policy'] = policy

    out = {}
    out['finite'] = elements.tree.flatdict(jax.tree.map(
        lambda x: jnp.isfinite(x).all(range(1, x.ndim)),
        dict(obs=obs, carry=carry, tokens=tokens, feat=feat, act=act)))
    carry = (enc_carry, dyn_carry, dec_carry, act)
    return carry, act, out, activations

In [18]:
def get_activations(obs, carry, agent):
    carry, act, out = agent.policy(carry, obs)
    deter = out['dyn/deter']
    stoch = out['dyn/stoch']

    # # change key names
    # out['deter'] = out['dyn/deter']
    # out['stoch'] = out['dyn/stoch']

    # feat2tensor = agent.model.feat2tensor
    # policy = agent.model.pol(feat2tensor(out), bdims=1)

    return carry, agent, {'deter': deter, 'stoch': stoch}

# Initialization

In [11]:
config = load_config()

# environment
SEED = 0
TASK = "reward"
INDEX = 0
seed = hash((SEED, INDEX)) % (2 ** 32 - 1)
env = crafter.Crafter(seed=seed, task=TASK)
  # adaptation from dreamerv3/main.py
notlog = lambda k: not k.startswith('log/')
obs_space = {k: v for k, v in env.obs_space.items() if notlog(k)}
act_space = {k: v for k, v in env.act_space.items() if k != 'reset'}

env.close()

# agent
# adaptation from dreamerv3/main.py
config = elements.Config(
      **config.agent,
      logdir=config.logdir,
      seed=config.seed,
      jax=config.jax,
      batch_size=config.batch_size,
      batch_length=config.batch_length,
      replay_context=config.replay_context,
      report_length=config.report_length,
      replica=config.replica,
      replicas=config.replicas,
  )
agentori = AgentOri(obs_space, act_space, config)

Replica: 0 / 1
Logdir: /workspace/assets/logdir/validation/20250121T195445
Run script: test
Observations
  image            Space(uint8, shape=(64, 64, 3), low=0, high=255)
  reward           Space(float32, shape=(), low=-inf, high=inf)
  is_first         Space(bool, shape=(), low=False, high=True)
  is_last          Space(bool, shape=(), low=False, high=True)
  is_terminal      Space(bool, shape=(), low=False, high=True)
Actions
  action           Space(int32, shape=(), low=0, high=17)
Extras
  consec           Space(int32, shape=(), low=-2147483648, high=2147483647)
  stepid           Space(uint8, shape=(20,), low=0, high=255)
  dyn/deter        Space(float32, shape=(8192,), low=-inf, high=inf)
  dyn/stoch        Space(float32, shape=(32, 64), low=-inf, high=inf)
JAX devices (1): [cuda:0]
Policy devices: cuda:0
Train devices:  cuda:0
Initializing parameters...
Optimizer opt has 165,965,075 params:
    95,495,168 dyn
    20,282,115 dec
    12,850,431 val
    12,606,481 pol
    10,749,

In [12]:
# agentori = overwrite_func(agentori)

# pm, ps = agentori.policy_mirrored, agentori.policy_sharded
# tp, pp = agentori.train_params_sharding, agentori.policy_params_sharding
# _, ar = agentori.partition_rules
# shared_kwargs = {'use_shardmap': agentori.jaxcfg.use_shardmap}

# agentori._policy = transform.apply(
#     nj.pure(agentori.model.policy), agentori.policy_mesh,
#     (pp, pm, ps, ps), (ps, ps, ps), ar,
#     static_argnums=(4,), **shared_kwargs)

TEST = "./models/agent.pkl"
agentori = load_agent(agentori, TEST)

# agent = agentori.model

Agent loaded from models/agent.pkl
Agent type: <class 'dict'>
<class 'embodied.jax.agent.Agent'>
<class 'embodied.jax.agent.Agent'>


In [ ]:
# if dataset (stats.jsonl files) have not been migrated to /dataset/, please run this
!python utils.py

In [13]:
# dataset
img_pos, img_epi = load_dataset("./dataset/crafter", pickle=False)
print(len(img_pos), len(img_epi))
assert len(img_pos) == len(img_epi)

BSIZE = config.batch_size
dl_space = DataLoader(img_pos, batch_size=BSIZE, shuffle=True)
dl_time = DataLoader(img_epi, batch_size=BSIZE, shuffle=True)

stats files num:  146


100%|██████████| 146/146 [02:37<00:00,  1.08s/it]

total length:  15849
15849 15849


In [23]:
import pickle

path = Path('./dataset/crafter/pickles')
os.makedirs(path, exist_ok=True)

with open(path/'img_pos.pkl', 'wb') as handle:
    pickle.dump(img_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path/'img_epi.pkl', 'wb') as handle:
    pickle.dump(img_epi, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
path_to_pkl="./models/agent.pkl"
path_to_pkl = Path(path_to_pkl)
with open(path_to_pkl, 'rb') as f:
    if "agent" in str(path_to_pkl):
        weight = pickle.load(f)
    else:
        weight = pickle.load(f)['agent']
print("Agent loaded from", path_to_pkl)
print("Agent type:", type(weight))
print(weight.keys())

Agent loaded from models/agent.pkl
Agent type: <class 'dict'>
dict_keys(['params', 'counters'])


In [15]:
print(weight['counters'])

{'updates': 50919, 'batches': 50924, 'actions': 1630370}


# Activations

In [16]:
jax.config.update("jax_transfer_guard", "allow")
from ninjax import pure

## spatial activations

In [19]:
carry = agentori.init_policy(batch_size=BSIZE)

path_to_save = Path('./dataset/act/')
os.makedirs(path_to_save, exist_ok=True)

images = []
positions = []
deters = []
stochs = []
logits = []
encoders = []
decoders = []
policies = []

dtype=jnp.uint8

cpu = lambda x: jax.device_put(x, device=jax.devices('cpu')[0]).astype(dtype)
cuda = lambda x: jax.device_put(x, device=jax.devices()[0]).astype(dtype)


for idx, batch in enumerate(tqdm.tqdm(dl_space)):
    if batch['image'].shape[0] != BSIZE:
        continue
    img = jnp.asarray(batch['image'].numpy())
    pos = jnp.asarray(batch['pos'].numpy())
    img = cuda(img)
    pos = cuda(pos)
    # img = jnp.asarray(img)
    # pos = jnp.asarray(pos)

    img = sg(img)
    pos = sg(pos)

    carry = sg(carry)
    obs = reconstruct_obs_dummy(img, BSIZE)

    carry, agentori, activations = get_activations(obs, carry, agentori)

    deter = activations['deter']
    stoch = activations['stoch']

    deter = np.array(sg(deter))
    stoch = np.array(sg(stoch))
    
    img = np.array(img)
    pos = np.array(pos)


    # logit = np.array(sg(logit))
    # enc = np.array(sg(activations['tokens']))
    # pol = np.array(sg(activations['policy']['action']))
    images.append(img)
    positions.append(pos)
    deters.append(deter)
    stochs.append(stoch)
    # logits.append(logit)
    # encoders.append(enc)
    # policies.append(pol)

images = np.stack(images, axis=0)
positions = np.stack(positions, axis=0)
deters = np.stack(deters, axis=0)
stochs = np.stack(stochs, axis=0)
# logits = np.stack(logits, axis=0)
# encoders = np.stack(encoders, axis=0)
# policies = np.stack(policies, axis=0)

np.savez_compressed(
    path_to_save / f'space.npz',
    images=images,
    positions=positions,
    deters=deters,
    stochs=stochs,
    # logits=logits,
    # encoders=encoders,
    # policies=policies
)

print("saved")

100%|██████████| 991/991 [00:09<00:00, 105.34it/s]


saved


## temporal activationspositions

In [21]:
carry = agentori.init_policy(batch_size=BSIZE)

path_to_save = Path('./dataset/act/')
os.makedirs(path_to_save, exist_ok=True)

images = []
episodes = []
deters = []
stochs = []
logits = []
encoders = []
decoders = []
policies = []

dtype=jnp.uint8

cpu = lambda x: jax.device_put(x, device=jax.devices('cpu')[0]).astype(dtype)
cuda = lambda x: jax.device_put(x, device=jax.devices()[0]).astype(dtype)


for idx, batch in enumerate(tqdm.tqdm(dl_time)):
    if batch['image'].shape[0] != BSIZE:
        continue
    img = jnp.asarray(batch['image'].numpy())
    epi = jnp.asarray(batch['episode'].numpy())
    img = cuda(img)
    epi = cuda(epi)
    # img = jnp.asarray(img)
    # pos = jnp.asarray(pos)

    img = sg(img)
    epi = sg(epi)

    carry = sg(carry)
    obs = reconstruct_obs_dummy(img, BSIZE)

    carry, agentori, activations = get_activations(obs, carry, agentori)

    deter = activations['deter']
    stoch = activations['stoch']

    deter = np.array(sg(deter))
    stoch = np.array(sg(stoch))
    
    img = np.array(img)
    epi = np.array(epi)


    # logit = np.array(sg(logit))
    # enc = np.array(sg(activations['tokens']))
    # pol = np.array(sg(activations['policy']['action']))
    images.append(img)
    episodes.append(epi)
    deters.append(deter)
    stochs.append(stoch)
    # logits.append(logit)
    # encoders.append(enc)
    # policies.append(pol)

images = np.stack(images, axis=0)
episodes = np.stack(episodes, axis=0)
deters = np.stack(deters, axis=0)
stochs = np.stack(stochs, axis=0)
# logits = np.stack(logits, axis=0)
# encoders = np.stack(encoders, axis=0)
# policies = np.stack(policies, axis=0)

np.savez_compressed(
    path_to_save / f'time.npz',
    images=images,
    episodes=episodes,
    deters=deters,
    stochs=stochs,
    # logits=logits,
    # encoders=encoders,
    # policies=policies
)

print("saved")

100%|██████████| 991/991 [00:09<00:00, 105.24it/s]


saved
